In [166]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from datetime import datetime
import time
import json
import re
import os

from langdetect import detect
import translators.server as ts

from tqdm import tqdm
from json import JSONDecodeError
import warnings
warnings.filterwarnings("ignore")

In [167]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

'2.11.0'

In [168]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36', 'Accept-Language':'pt-BR,pt;q=0.9,en;q=0.8'}
s = requests.Session()

In [169]:
def get_data(url, classe_name, tag_name):
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, 'html.parser')
    texto = ''
    try:
        texto = soup.find_all(tag_name, class_= classe_name).text.split('\n')
        return texto
    
    except AttributeError:
        return []


In [401]:
input = 'GTX 1630'
text = input + ' amazon'
url = "https://www.google.com/search?q=" + text

request_result=s.get(url).content

soup = BeautifulSoup(request_result,"html.parser")

In [402]:
cont = 0

new_url = ''
reviews = []
produtos = []
for a in soup.find_all('a', href=True):    
    if 'www.amazon.com' in a['href'] and '/dp/' in a['href']:
        href888 = a['href']
        produtos.append(a.h3.text)
        new_url = href888.replace('/url?q=', '') # remove /url?q=
        url_coments = new_url.split('&')[0].replace('/dp/', '/product-reviews/').replace('.br', '')
        if '//'  in url_coments:
            url_coments = url_coments.split('//')[1]
        reviews.append('https://' + url_coments)

In [418]:
from math import ceil
coments = []
cont=0
for r in reviews:
    
    try:
        html = s.get(r + f'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1', headers=headers).content
        soup = BeautifulSoup(html, 'html.parser')

        a_soupi = soup.find_all('div', class_='a-row a-spacing-base a-size-base')
        
        if len(a_soupi) == 0:
            continue
        print(f"Analisando produto {produtos[cont]}")
        print(a_soupi[0].text.replace('\n', '').replace('  ', ''))
        i = int(a_soupi[0].text.replace('\n', '').replace('  ', '').replace(',', '').split('ratings ')[1].split(' ')[0])/10
        i = ceil(i)


        print(f"{i} Comentários\n")

        for j in range(1, i):
            html = s.get(r + f'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber={j}', headers=headers).content
            a_soup = soup.find_all('div', class_='a-row a-spacing-small review-data')
            print(a_soup)
            
            for a in a_soup:
                for coment in a.text.split('\n'):
                    if coment != '':
                        coments.append(coment)
        
    except:
        print('error')
        continue

    cont+=1

Analisando produto MSI Gaming GeForce GTX 1630 4GB GDRR6 64-Bit HDMI/DP/DVI ...
4 total ratings, 2 with reviews
1 Comentários

Analisando produto EVGA GeForce GTX 1630 SC Gaming, 04G-P4-1633-KR, 4GB ...
2 total ratings, 2 with reviews
1 Comentários



[]

In [404]:
coments_translate = []
for c in coments:
    try:
        detec = detect(c)
        if detec != 'en':
            len(c)
            coments_translate.append(ts.google(c, to_language='en'))
        else:
            coments_translate.append(c)
    except:
        try:
            coments_translate.append(ts.google(c, to_language='en'))
        except:
            
            continue
        continue

In [405]:
print(f'{len(coments_translate)} de {len(coments)} comentário traduzidos válidos')

0 de 0 comentário traduzidos válidos


In [406]:
coments

[]

## Testando modelo

In [407]:
from sklearn.linear_model import LogisticRegression
import nltk

In [408]:
# importando a lista da nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords_en = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pichau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [409]:
import spacy.cli

spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [410]:
import en_core_web_sm

spc_en = en_core_web_sm.load()

In [411]:
df_alexa = pd.read_csv('amazon_alexa.tsv', sep = '\t')

In [412]:
positive = df_alexa[df_alexa['feedback'] == 1]
negative = df_alexa[df_alexa['feedback'] == 0]


In [413]:
df_alexa.drop(['date', 'rating', 'variation'], 1, inplace=True)


In [414]:
df_alexa

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1
...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",1
3146,"Listening to music, searching locations, check...",1
3147,"I do love these things, i have them running my...",1
3148,Only complaint I have is that the sound qualit...,1


In [415]:
def limpa_texto(texto):
  '''(str) -> str
  Essa funcao recebe uma string, deixa tudo em minusculo, filtra apenas letras,
  retira stopwords, lemmatiza e retorna a string resultante.
  '''
  texto = texto.lower()

  texto = re.sub(r"[\W\d_]+", " ", texto)

  texto = [pal for pal in texto.split() if pal not in stopwords_en]

  spc_texto = spc_en(" ".join(texto))
  tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto]
  
  return " ".join(tokens)

In [416]:
# Aplica a funcao nas reviews do dataset
df_alexa['verified_reviews'] = df_alexa['verified_reviews'].apply(limpa_texto)

KeyboardInterrupt: 

## Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
texto = df_alexa['verified_reviews']
# Importando o TfidfVectorizer

# Instanciando o TfidfVectorizer
tfidf_vect = TfidfVectorizer()

# Vetorizando
X_tfidf = tfidf_vect.fit_transform(texto)

## Modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf.toarray(), df_alexa['feedback'], test_size = 0.2, random_state = 42)

In [ ]:
X_train.shape


(2520, 3103)

In [ ]:
X_test.shape

(630, 3103)

In [ ]:
# reglog = LogisticRegression()

# # Aplicando o modelo
# reglog.fit(X_train, y_train)

# # Predicao
# y_pred_test = reglog.predict(X_test)

classifier = tf.keras.models.Sequential()
classifier.add(tf.keras.layers.Dense(units = 10, activation='relu', input_shape=(3103,)))
classifier.add(tf.keras.layers.Dense(units = 10, activation='relu'))
classifier.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

In [ ]:
classifier.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])


In [ ]:
epochs_hist = classifier.fit(X_train, y_train, epochs=10)


Epoch 1/10
79/79 [==============================] - 1s 1ms/step - loss: 0.5951 - accuracy: 0.9036
Epoch 2/10
79/79 [==============================] - 0s 1ms/step - loss: 0.3462 - accuracy: 0.9210
Epoch 3/10
79/79 [==============================] - 0s 1ms/step - loss: 0.2459 - accuracy: 0.9210
Epoch 4/10
79/79 [==============================] - 0s 1ms/step - loss: 0.2155 - accuracy: 0.9210
Epoch 5/10
79/79 [==============================] - 0s 1ms/step - loss: 0.1827 - accuracy: 0.9210
Epoch 6/10
79/79 [==============================] - 0s 1ms/step - loss: 0.1421 - accuracy: 0.9381
Epoch 7/10
79/79 [==============================] - 0s 1ms/step - loss: 0.1086 - accuracy: 0.9595
Epoch 8/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0833 - accuracy: 0.9746
Epoch 9/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0660 - accuracy: 0.9806
Epoch 10/10
79/79 [==============================] - 0s 1ms/step - loss: 0.0538 - accuracy: 0.9865


In [ ]:
y_pred_test = (classifier.predict(X_test) > 0.5).astype(int)
# y_pred_test = (y_pred_test > 0.5).astype(int)

20/20 [==============================] - 0s 763us/step


In [ ]:
print(classification_report(y_test,y_pred_test))

              precision    recall  f1-score   support

           0       0.80      0.41      0.55        58
           1       0.94      0.99      0.97       572

    accuracy                           0.94       630
   macro avg       0.87      0.70      0.76       630
weighted avg       0.93      0.94      0.93       630



In [ ]:
texto_vetorizado = tfidf_vect.transform(['horrible product'])
y_pred_test = (classifier.predict(texto_vetorizado.toarray()) > 0.5).astype(int)
y_pred_test

1/1 [==============================] - 0s 18ms/step


array([[0]])

In [ ]:
coments_translate

['Edit: Customer Service swapped my flawed GPU with a new in box video card.It works well, 85fps in Cinebench. But it does have two issues, and they are tenable. The four rear outputs are mislabeled. I have a programs that looks for device default audio on "Playback Device Digital Audio 1". I had to put the cable in port 3 to make it do that in the sound properties.The fan on the GPU is erratic. I used Speedfan to work around. On the "Fan Control Tab" I made a "Fan Controller" and used GPU Fan from ATI Video Card as the "Controlled Speed". With my 50MM helper fan controlled by the CPU Temp, the temps stay between 26C and 53C.About one year ago I bought my first USED peripheral from Amazon, a MSi RX560 4GB that still works great. So when I discovered that the New VisionTek RX560 4GB LP I ordered was fulfilled with an open box instead. I didn\'t panic, because I always have a great experience with Amazon.How an open box got in with the New products, I will never know. The Amazon CS seeme

In [ ]:
c = tfidf_vect.transform(coments_translate).toarray()
predict = (classifier.predict(c) > 0.5).astype(int)

3/3 [==============================] - 0s 1ms/step


In [ ]:
cont_pos = 0
cont_neg = 0
for p in predict:
    if p == 1:
        cont_pos += 1
    else:
        cont_neg += 1

In [ ]:
print(f'Positivos: {cont_pos} | Negativos: {cont_neg}')

Positivos: 72 | Negativos: 5
